# Train a Simple Audio Recognition model for microcontroller use

This notebook demonstrates how to train a 20kb [Simple Audio Recognition](https://www.tensorflow.org/tutorials/sequences/audio_recognition) model for [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview). It will produce the same model used in the [micro_speech](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/experimental/micro/examples/micro_speech) example application.

The model is designed to be used with [Google Colaboratory](https://colab.research.google.com).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/experimental/micro/examples/micro_speech/train_speech_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/experimental/micro/examples/micro_speech/train_speech_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


The notebook runs Python scripts to train and freeze the model, and uses the TensorFlow Lite converter to convert it for use with TensorFlow Lite for Microcontrollers.

**Training is much faster using GPU acceleration.** Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and selecting **GPU**. Training 18,000 iterations will take 1.5-2 hours on a GPU runtime.

## Configure training

The following `os.environ` lines can be customized to set the words that will be trained for, and the steps and learning rate of the training. The default values will result in the same model that is used in the micro_speech example. Run the cell to set the configuration:

###install conda env used .yaml <br>
---> [url](https://github.com/Choitord/TF1.14_conda_env.git) <---

## env<br/>
TensorFlow1.14.0<br>
python3.7.x<br>

In [1]:
import os

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All other words will be used to train an "unknown" category.
os.environ["WANTED_WORDS"] = "on,off"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=15000,3000 and LEARNING_RATE=0.001,0.0001
# will run 18,000 training loops in total, with a rate of 0.001 for the first
# 15,000, and 0.0001 for the final 3,000.
os.environ["TRAINING_STEPS"]="15000,3000"
os.environ["LEARNING_RATE"]="0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
total_steps = sum(map(lambda string: int(string),
                  os.environ["TRAINING_STEPS"].split(",")))
os.environ["TOTAL_STEPS"] = str(total_steps)

# Print the configuration to confirm it
'''
!echo "Training these words: ${WANTED_WORDS}"
!echo "Training steps in each stage: ${TRAINING_STEPS}"
!echo "Learning rate in each stage: ${LEARNING_RATE}"
!echo "Total number of training steps: ${TOTAL_STEPS}"
'''
print("Training these words:", os.environ["WANTED_WORDS"])
print("Training steps in each stage:", os.environ["TRAINING_STEPS"])
print("Learning rate in each stage:", os.environ["LEARNING_RATE"])
print("Total number of training steps:", os.environ["TOTAL_STEPS"])


Training these words: on,off
Training steps in each stage: 15000,3000
Learning rate in each stage: 0.001,0.0001
Total number of training steps: 18000


## Install dependencies

Next, we'll install a GPU build of TensorFlow, so we can use GPU acceleration for training.

In [0]:
# Replace Colab's default TensorFlow install with a more recent
# build that contains the operations that are needed for training
#!pip uninstall -y tensorflow tensorflow_estimator tensorboard
#!pip install -q tf-estimator-nightly==1.14.0.dev2019072901 tf-nightly-gpu==1.15.0.dev20190729

We'll also clone the TensorFlow repository, which contains the scripts that train and freeze the model.

In [2]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C tensorflow checkout 17ce384df70

fatal: destination path 'tensorflow' already exists and is not an empty directory.


M	tensorflow/examples/speech_commands/train.py


HEAD is now at 17ce384df70 Share ownership of `UnboundedWorkQueue` between collective executor and executor manager.


## Load TensorBoard

Now, set up TensorBoard so that we can graph our accuracy and loss as training proceeds.

In [5]:
#!pip uninstall tensorboard
!pip install tensorboard

In [6]:
!pip show tensorboard

Name: tensorboard
Version: 1.14.0
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\phant\anaconda3\envs\tf1\lib\site-packages
Requires: absl-py, grpcio, markdown, numpy, protobuf, setuptools, six, werkzeug, wheel
Required-by: tensorflow, tensorflow-gpu


## Begin training

Next, run the following script to begin training. The script will first download the training data:

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    

'''
##notion## 

# if using RTX30XX or Higher  TensorFlow 1.x not support GPU use  (ex. TensorFlow1.14.0 RTX3060 12GB => RTX30xx support Cuda11.x ver. TensorFlow1.x version not support Cuda11.x )
But if you must use TF1.x & RTX30xx(or Higher) , then build own TF1.x use docker, wsl, etc..
"-1" option for only use CPU 
" 0" option for use GPU 
if Cudatoolkit & Cudnn was not installed in env GPU didnt load

test env used RTX3060 so defalt was "-1", if used RXT1050 changed option to "0"

train time(CPU):64m24s(AMD 9600X) 

'''

'\n##notion## \n\n# if using RTX30XX or Higher  TensorFlow 1.x not support GPU use  (ex. TensorFlow1.14.0 RTX3060 12GB => RTX30xx support Cuda11.x ver. TensorFlow1.x version not support Cuda11.x )\nBut if you must use TF1.x & RTX30xx(or Higher) , then build own TF1.x use docker, wsl, etc..\n"-1" option for only use CPU \n" 0" option for use GPU \nif Cudatoolkit & Cudnn was not installed in env GPU didnt load\n\ntest env used RTX3060 so defalt was "-1", if used RXT1050 changed option to "0"\n\ntrain time(CPU):64m24s(AMD 9600X) \n\n'

In [3]:
!python tensorflow/tensorflow/examples/speech_commands/train.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=%WANTED_WORDS% --silence_percentage=25 --unknown_percentage=25 \
--quantize=1 --verbosity=WARN --how_many_training_steps=%TRAINING_STEPS% \
--learning_rate=%LEARNING_RATE% --summaries_dir=/content/retrain_logs \
--data_dir=/content/data --train_dir=/content/train \


python: can't open file 'tensorflow/tensorflow/examples/speech_commands/train.py': [Errno 2] No such file or directory


In [ ]:
import tensorflow as tf
print(hasattr(tf.contrib, 'quantize'))  # True/False 확인

## Freeze the graph

Once training is complete, run the following cell to freeze the graph.

In [35]:
!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=mfcc \
--wanted_words=%WANTED_WORDS% --quantize=1 --output_file=/content/tiny_conv.pb \
--start_checkpoint=/content/train/tiny_conv.ckpt-%TOTAL_STEPS%

c:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework

## Convert the model

Run this cell to use the TensorFlow Lite converter to convert the frozen graph into the TensorFlow Lite format, fully quantized for use with embedded devices.

In [39]:
!toco \
--graph_def_file=/content/tiny_conv.pb --output_file=/content/tiny_conv.tflite \
--input_shapes=1,4000 --input_arrays=Reshape_2 --output_arrays=labels_softmax \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

C:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\phant\anaconda3\envs\TF1\lib\site-packages\tensorflow\python\framework

The following cell will print the model size, which will be under 20 kilobytes.

In [40]:
import os
model_size = os.path.getsize("/content/tiny_conv.tflite")
print("Model is %d bytes" % model_size)

Model is 17032 bytes


Finally, we use xxd to transform the model into a source file that can be included in a C++ project and loaded by TensorFlow Lite for Microcontrollers.

In [41]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i /content/tiny_conv.tflite > /content/tiny_conv.cc
# Print the source file
!cat /content/tiny_conv.cc

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


'cat'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
